# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file= "../../../weather.csv"
weather= pd.read_csv(weather_file)
weather_df=pd.DataFrame(weather)
weather_df.head()


,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Clouds,Wind Speed
0,0,ostrovnoy,68.05,39.51,53.04,85,44,10.42
1,1,mahebourg,-20.41,57.70,69.80,94,75,3.36
2,2,qaanaaq,77.48,-69.36,40.59,66,0,4.36
3,3,zhuhai,22.28,113.57,79.52,100,40,14.99
4,4,cape town,-33.93,18.42,61.45,82,0,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
fig=gmaps.figure()

heatmap_layer=gmaps.heatmap_layer(
    weather_df[["Latitude", "Longitude"]], weights=weather_df["Humidity"], max_intensity=100, point_radius=5, dissipating=False)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_temp= weather_df.loc[(weather_df["Temperature"]>=72) & (weather_df["Temperature"]<=78)]
ideal_temp

ideal_humid= ideal_temp.loc[ideal_temp["Humidity"]<60]
ideal_humid

ideal_weather= ideal_humid.loc[(ideal_humid["Clouds"]<30) & (ideal_humid["Clouds"]>10)]
ideal_weather

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Clouds,Wind Speed
210,210,ribeira grande,38.52,-28.70,77.00,50,20,8.05
261,261,fort frances,48.60,-93.40,75.20,50,20,4.70
276,276,rabo de peixe,37.80,-25.58,77.00,53,20,13.87
304,304,kvarkeno,52.08,59.73,74.01,47,28,8.72
363,363,vila franca do campo,37.72,-25.43,77.00,53,20,13.87
437,437,sioux lookout,50.10,-91.92,75.20,53,20,5.82
470,470,horta,38.53,-28.63,77.00,50,20,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=pd.DataFrame()
hotel_df["Hotel Name"]=""
    
hotel_df["Latitude"]=ideal_weather["Latitude"]
hotel_df["Longitude"]=ideal_weather["Longitude"]
hotel_df["City"]=ideal_weather["City"]
hotel_df.reset_index(drop=True, inplace=True)
country=["Portugal", "Canada", "Portugal", "Russia", "Portugal", "Canada", "Portugal" ]
hotel_df["Country"]=country

hotel_df

,Hotel Name,Latitude,Longitude,City,Country
0,NaN,38.52,-28.70,ribeira grande,Portugal
1,NaN,48.60,-93.40,fort frances,Canada
2,NaN,37.80,-25.58,rabo de peixe,Portugal
3,NaN,52.08,59.73,kvarkeno,Russia
4,NaN,37.72,-25.43,vila franca do campo,Portugal
5,NaN,50.10,-91.92,sioux lookout,Canada
6,NaN,38.53,-28.63,horta,Portugal


In [7]:
lats=[]
lngs=[]

for each in range(len(hotel_df["City"])):
    lats.append(hotel_df["Latitude"][each])
    lngs.append(hotel_df["Longitude"][each])


hotels=[]

for each in range(len(hotel_df["City"])):
    url= f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?type=establishment&keyword=hotel&fields=name&key={g_key}&radius=5000&location="
    query_url=url+str(lats[each])+","+str(lngs[each])
    response=requests.get(query_url).json()
    hotels.append(response["results"][0]["name"])
    

hotel_df["Hotel Name"]=hotels

hotel_df.to_csv(r"C:\Users\chris\Documents\DU Bootcamp\Class_Materials\06-Python-APIs\Homework\Instructions\starter_code\Hotels.csv")
hotel_df

,Hotel Name,Latitude,Longitude,City,Country
0,Quinta Das Buganvílias,38.52,-28.70,ribeira grande,Portugal
1,AmericInn by Wyndham International Falls,48.60,-93.40,fort frances,Canada
2,Pedras do Mar Resort & SPA,37.80,-25.58,rabo de peixe,Portugal
3,"Ooo ""Kvarkeno""",52.08,59.73,kvarkeno,Russia
4,Pestana Bahia Praia,37.72,-25.43,vila franca do campo,Portugal
5,Sunset Inn & Suites,50.10,-91.92,sioux lookout,Canada
6,Hotel Horta,38.53,-28.63,horta,Portugal


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Latitude", "Longitude"]]
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))